In [ ]:
import json
import pandas as pd
import numpy as np
import re

In [ ]:
file_dir = 'wikipedia-movies.json'
file_dir

In [ ]:
with open(file_dir, mode='r') as file:
    wiki_movies_raw = json.load(file)

In [ ]:
# First 5 records
wiki_movies_raw[:5]

In [ ]:
# Last 5 records
wiki_movies_raw[-5:]

In [ ]:
# Some records in the middle
wiki_movies_raw[3600:3605]

In [ ]:
file_dir = 'Movies_ETL/Data/movies_metadata.csv'
file_dir

In [ ]:
file_dir1 = 'Movies_ETL/Data/ratings.csv'
file_dir1

In [ ]:
kaggle_metadata = pd.read_csv(file_dir, low_memory=False)
ratings = pd.read_csv('ratings.csv')

In [ ]:
ratings.head()

In [ ]:
kaggle_metadata.head()

In [ ]:
kaggle_metadata.tail()

In [ ]:
kaggle_metadata.sample(5)

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.head(5)


In [ ]:
wiki_movies_df.columns.tolist()

In [ ]:
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie]
len(wiki_movies)

In [ ]:
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

In [ ]:
len(wiki_movies)

In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    return movie

In [ ]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

In [ ]:
sorted(wiki_movies_df.columns.tolist())

In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    return movie

In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:

    return movie

In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
        

    return movie

In [ ]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]


In [ ]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [ ]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

In [ ]:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()

In [ ]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [ ]:
box_office = wiki_movies_df['Box office'].dropna()
len(box_office)

In [ ]:
def is_not_a_string(x):
    return type(x) != str

In [ ]:
box_office[box_office.map(is_not_a_string)]

In [ ]:
#  lambada code below id does the same process as the function def is_not_a_string
box_office[box_office.map(lambda x: type(x) != str)]

In [ ]:
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
box_office

In [ ]:
form_one = r'\$\d+\.?\d*\s*[mb]illion'
box_office.str.contains(form_one, flags=re.IGNORECASE).sum()

In [ ]:
form_two = r'\$\d{1,3}(?:,\d{3})+'
box_office.str.contains(form_two, flags=re.IGNORECASE).sum()


In [ ]:
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE)

In [ ]:
box_office[~matches_form_one & ~matches_form_two]

In [ ]:
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [ ]:
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
wiki_movies_df['box_office']

In [ ]:
wiki_movies_df.drop('Box office', axis=1, inplace=True)

In [ ]:
budget = wiki_movies_df['Budget'].dropna()

In [ ]:
#convert any list to srings
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
budget

In [ ]:
#remove any values between a dollar sign and a hyphen (for budgets given in ranges):
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
form_one = r'\$\d+\.?\d*\s*[mb]illion'
budget.str.contains(form_one, flags=re.IGNORECASE).sum()

In [ ]:
form_two = r'\$\d{1,3}(?:,\d{3})+'
budget.str.contains(form_two, flags=re.IGNORECASE).sum()


In [ ]:
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE)
budget[~matches_form_one & ~matches_form_two]

In [ ]:
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matches_form_one & ~matches_form_two]

In [ ]:
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [ ]:
wiki_movies_df.drop('Budget', axis=1, inplace=True)

In [ ]:
release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)


In [ ]:
#Full month name, one- to two-digit day, four-digit year (i.e., January 1, 2000)
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
#Four-digit year, two-digit month, two-digit day, with any separator (i.e., 2000-01-01)
date_form_two = r'\d{4}.[01]\d.[123]\d'
#Full month name, four-digit year (i.e., January 2000)
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
# Four-digit year
date_form_four = r'\d{4}'

In [ ]:
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

In [ ]:
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

In [ ]:
#   PARSE RUNNING TIME

In [ ]:
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
# using string boundaries count the different running time formats in the file 
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE).sum()

In [ ]:
#check running time formats of the remaing records 
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE) != True]

In [ ]:
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE).sum()

In [ ]:
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE) != True]

In [ ]:
# extract digits for both possible patterns. 
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

In [ ]:
#convert string to numericc and change all NaNs to zero using fillna()
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

In [ ]:
# convert hour capture groups and minute capture groups to minutes and save the output to wiki_movies_df, if the pure minutes capture group is zero 
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

In [ ]:
wiki_movies_df.drop('Running time', axis=1, inplace=True)

In [ ]:
# check Kaggle data types
kaggle_metadata.dtypes

In [ ]:
kaggle_metadata['adult'].value_counts()

In [ ]:
kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]

In [ ]:
# keep rows where the adult column is False, and then drop the adult column.
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')

In [ ]:
# Inspect video column
kaggle_metadata['video'].value_counts()

In [ ]:
# create the Boolean column and assign it back to "video" 
kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'

In [ ]:
# convert numeric column to numeric data tyoes set errors = 'raise' to flag data that could not be converted to numeric
kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

In [ ]:
# convert release_date to datetime
kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

In [ ]:
# Reasonability Checks on Ratings Data

In [ ]:
ratings.info(null_counts=True)

In [ ]:
# specify in to_datetime() that the origin is 'unix' and the time unit is seconds.
pd.to_datetime(ratings['timestamp'], unit='s')


In [ ]:
# replace timestamp column with calculated time using Unix epoch
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [ ]:
#statistics of the actual ratings to check for  errors using histogram
pd.options.display.float_format = '{:20,.2f}'.format
ratings['rating'].plot(kind='hist')
ratings['rating'].describe()